создаем файл с числами

In [ ]:
import random

# Открываем файл для записи
with open("random_numbers.txt", "w") as file:
    # Генерируем и записываем 50000 случайных 32-битных целых чисел
    for _ in range(50000):
        random_number = random.randint(2, 2**31 - 1)
        file.write(str(random_number) + "\n")

простой последовательный алгоритм 

In [10]:
import math
import time
# Функция для подсчета простых множителей числа

def count_prime_factors(num):
    count = 0
    # Проверяем делители 
    for i in range(2, int(math.sqrt(num)) + 1):
        while num % i == 0:
            print(num, i, count)
            count += 1
            num //= i
    return count + (num > 1)

start_time = time.time()
print(start_time)
# Считываем числа из файла и подсчитываем простые множители для каждого числа
total_prime_factors = 0

with open("random_numbers.txt", "r") as file:
    for line in file:
        number = int(line.strip())
        total_prime_factors += count_prime_factors(number)

end_time = time.time()
print(end_time)
res = end_time - start_time
print("Суммарное количество простых множителей для чисел из файла:", total_prime_factors)
print(f"Время выполнения {res:.1f} секунд")

1714650482.524309
num: 1
num: 223253
num: 1
num: 33857
num: 4615549
num: 82793
num: 1
num: 531106313
num: 10418281
num: 553013
num: 485447
num: 355519
num: 56437
num: 262079
num: 2697587
num: 93239
num: 870323
num: 73681
num: 3565747
num: 25090717
num: 132859
num: 139273
num: 3353107
num: 2517509
num: 10789
num: 1
num: 12601
num: 20627
num: 1
num: 6747509
num: 1
num: 1397867327
num: 673397
num: 219476317
num: 117659611
num: 346207
num: 632227
num: 59791
num: 7244179
num: 1
num: 1759991
num: 1539907507
num: 4767527
num: 2686477
num: 784351
num: 680440433
num: 1
num: 54251
num: 163019
num: 1
num: 118423
num: 96157
num: 1
num: 1
num: 52415171
num: 1351783
num: 103958731
num: 148913551
num: 1
num: 2635253
num: 1
num: 104202467
num: 14132929
num: 18643007
num: 1126771
num: 2289649
num: 241190591
num: 5521249
num: 1123081
num: 144435821
num: 1
num: 7819391
num: 944701
num: 7739903
num: 136879
num: 902191
num: 1
num: 9381503
num: 131837
num: 2084111
num: 225746341
num: 3906373
num: 96181
num:

мультипроцессинг 

In [7]:
import math
import multiprocessing
import time

start_time = time.time()

# Функция для подсчета простых множителей числа
def count_prime_factors(num):
    count = 0
    # Проверяем делители 
    for i in range(2, int(math.sqrt(num)) + 1):
        while num % i == 0:
            count += 1
            num //= i
    return count + (num > 1)

# Функция для обработки данных в отдельном процессе
def process_data(chunk):
    total_prime_factors = sum(count_prime_factors(num) for num in chunk)
    return total_prime_factors

# Открываем файл для чтения и считываем числа
numbers = []
with open("random_numbers.txt", "r") as file:
    for line in file:
        numbers.append(int(line.strip()))

# Подготовка данных для распределения по процессам
chunk_size = len(numbers) // multiprocessing.cpu_count()
chunks = [numbers[i:i + chunk_size] for i in range(0, len(numbers), chunk_size)]

# Создание процессов и запуск обработки данных
results = []
for chunk in chunks:
    result = process_data(chunk)
    results.append(result)



# Суммируем результаты из всех процессов
total_prime_factors = sum(results)

end_time = time.time()
res = end_time - start_time

print("Суммарное количество простых множителей для чисел из файла:", total_prime_factors)
print(f"Время выполнения {res:.1f} секунд")


Суммарное количество простых множителей для чисел из файла: 203470
Время выполнения 78.5 секунд


с помощью Dask

In [8]:
import math
import dask.bag as db
import time

start_time = time.time()
# Функция для подсчета простых множителей числа
def count_prime_factors(num):
    count = 0
    # Проверяем делители 
    for i in range(2, int(math.sqrt(num)) + 1):
        while num % i == 0:
            count += 1
            num //= i
    return count + (num > 1)

# Чтение чисел из файла 
numbers_bag = db.read_text("random_numbers.txt").map(int)

# Подсчет простых множителей для каждого числа
prime_factors_count = numbers_bag.map(count_prime_factors).sum().compute()


end_time = time.time()
res = end_time - start_time
print("Суммарное количество простых множителей для чисел из файла:", prime_factors_count)
print(f"Время выполнения {res:.1f} секунд")

Суммарное количество простых множителей для чисел из файла: 203470
Время выполнения 76.6 секунд
